# LSTM을 이용하여 텍스트 생성하기

In [1]:
import pandas as pd
import numpy as np

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [ ]:
df = pd.read_csv('ArticlesApril2018.csv')
df.head()

In [ ]:
# headline에 결측치가 있는지 확인
print(df['headline'].isnull().sum())

In [ ]:
# 헤드라인의 값들을 리스트로 저장
hl = df['headline'].tolist()

In [ ]:
# 현재 샘플의 개수 출력
print(len(hl))

In [ ]:
# Unknown 값을 가진 샘플 제거
filtered_headline = []

for word in hl:
    if word !='Unknown':
        filtered_headline.append(word)

# Unknown 값 제거 후 샘플의 개수 출력
print(len(filtered_headline))

In [ ]:
# 구두점(!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~)불러오는 코드
from string import punctuation

# 데이터 전처리 함수
def repreprocessing(raw_sentence):
    # 아스키코드로 디코딩하는 과정에서 아스키 코드로 표현할 수 없는 문자는 사라짐
    preproceseed_sentence = raw_sentence.encode("utf8").decode("ascii",'ignore')
    # 구두점 제거 및 소문자화
    return ''.join(word for word in preproceseed_sentence if word not in punctuation).lower()

# filtered_headline 각 요소에 대해 데이터 전처리 과정 수행
preprocessed_headline = []
for i in filtered_headline:
    pre = repreprocessing(i)
    preprocessed_headline.append(pre)

# 전처리 완료된 새로운 리스트 생성
preprocessed_headline[:5]

In [ ]:
# 토큰화 및 단어 집합의 크기(vocab_size) 계산
tokenizer = Tokenizer()
tokenizer.fit_on_texts(preprocessed_headline)
vocab_size = len(tokenizer.word_index) + 1

In [ ]:
# 문장을 분해하여 sequence 생성
sequences = []

for sentence in preprocessed_headline:
    encoded = tokenizer.texts_to_sequences([sentence])[0]

    for i in range(1, len(encoded)):
        sequence = encoded[: i+1]
        sequences.append(sequence)
sequences[:15]

In [ ]:
# 샘플(sequence)의 최대 길이 구하기
max_len = max(len(l) for l in sequences)

lens = []
for line in sequences:
    lens.append(len(line))
max_len = max(lens)

In [ ]:
# 시퀀스 패딩
sequences = pad_sequences(sequences, maxlen=max_len)
sequences[:3]

In [14]:
# 시퀀스 넘파이 배열로 변환
sequences = np.array(sequences)

# 마지막 단어를 제외한 부분을 입력 데이터로 사용
X = sequences[:, :-1]

# 마지막 단어를 출력 데이터로 사용
y = sequences[:, -1]

In [ ]:
# data(X) 3행 출력
X[:3]

In [ ]:
# target(y) 3행 출력
y[:3]

In [17]:
# target(y)에 대해 원 핫 인코딩
y = to_categorical(y, num_classes=vocab_size)

In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM

In [20]:
# 모델 생성
model = Sequential()
model.add(Embedding(vocab_size, 64))
model.add(LSTM(128))
model.add(Dense(vocab_size, activation='softmax'))

# 모델 컴파일
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# 모델 학습
model.fit(X, y, epochs=100)

In [69]:
# 문장 생성기(RNN과 완전 동일)
# 모델, 토크나이저, 현재 단어, 반복할 횟수
def sentence_generation(model, tokenizer, current_word, n):
    init_word = current_word
    sentence = ''

    for _ in range(n):
        encoded = tokenizer.texts_to_sequences([current_word])[0]
        encoded = pad_sequences([encoded], maxlen=max_len-1, padding='pre')

        result = model.predict(encoded, verdose=0)
        result = np.argmax(result, axis=1)

        for word, index in tokenizer.word_index.items():
            if index == result:
                break

        current_word = current_word + ' ' + word

        sentence = sentence + ' ' + word

    sentence = init_word + sentence
    return sentence

In [ ]:
print(sentence_generation(model, tokenizer, 'i want', 10))

In [ ]:
print(sentence_generation(model, tokenizer, 'how', 10))